In [1]:
import numpy as np
import matplotlib.pyplot as plt


image_height = 48
image_width = 48
emotions_count = 8
emotion_labels = ['neutral', 'happiness', 'surprise', 'sadness', 'anger', 'disgust', 'fear', 'contempt']

samples = 35393 # 2~35394
training_samples = 28317  # 2~28318 (Training)
validation_samples = 3541 # 28319~31859 (PublicTest)
test_samples = 3535       # 31860~35394 (PrivateTest)

In [2]:
import tensorflow as tf
from tensorflow.python.keras.layers import Dense, GlobalAveragePooling2D, MaxPool2D, Input, Conv2D, Flatten
from tensorflow.python.keras.models import Model
from tensorflow.python.keras import layers, Sequential,losses, metrics
from tensorflow.python.keras import optimizers, callbacks, models
from tensorflow.python.keras.optimizer_v2 import adam

In [3]:
image_path = "./dataset/images.npy"
emotion_multi_path = "./dataset/emotions_multi.npy"
emotion_single_path = "./dataset/emotions_single.npy"

images = np.load(image_path)
emotions_multi = np.load(emotion_multi_path)
emotions_single = np.load(emotion_single_path)

print(images.shape)
print(emotions_multi.shape)
print(emotions_single.shape)

(35393, 48, 48, 1)
(35393, 8)
(35393, 8)


In [4]:
#emotions = emotions_single
emotions = emotions_multi

images = tf.convert_to_tensor(images)
#images = tf.image.grayscale_to_rgb(images)
emotions = tf.convert_to_tensor(emotions)
print("images shape:", images.shape)
print("emotions shape:", emotions.shape)

images shape: (35393, 48, 48, 1)
emotions shape: (35393, 8)


In [5]:
from tensorflow.python.keras import layers
# choose one method:
images = layers.Rescaling(1./127.5, offset= -1)(images)

In [6]:
training_size = training_samples + validation_samples
test_size = test_samples

training_images = images[:training_size]
test_images = images[training_size:]
training_emotions = emotions[:training_size]
test_emotions = emotions[training_size:]

print("training_images shape:", training_images.shape)
print("training_emotions shape:", training_emotions.shape)
print("test_images shape:", test_images.shape)
print("test_emotions shape:", test_emotions.shape)

training_images shape: (31858, 48, 48, 1)
training_emotions shape: (31858, 8)
test_images shape: (3535, 48, 48, 1)
test_emotions shape: (3535, 8)


In [7]:
from tensorflow.python.keras import losses, metrics
from tensorflow.python.keras.optimizer_v2 import adam

cce = losses.CategoricalCrossentropy()
mse = losses.MeanSquaredError()

tf.config.run_functions_eagerly(True)
def model_acc(y_true, y_pred):
    size = y_true.shape[0]
    acc = 0
    for i in range(size):
        true = y_true[i]
        pred = y_pred[i]           
        index_max = tf.argmax(pred).numpy()
        if true[index_max].numpy()==tf.reduce_max(true).numpy():
            acc += 1
    return acc/size

In [8]:
from tensorflow.python.keras.applications import vgg16, resnet_v2, densenet, efficientnet
from tensorflow.python.keras.layers import Dense, GlobalAveragePooling2D, MaxPool2D, Input, Conv2D, Flatten, Concatenate, Dropout
from tensorflow.python.keras.models import Model
from tensorflow.python.keras import layers, Sequential

# VGG13 combined .5dropout
input_layer = Input(shape=(48,48,3))
print(input_layer.shape)
feat1 = GlobalAveragePooling2D()(input_layer)
print("feature1", feat1.shape)
x = Conv2D (filters=64, kernel_size=3, padding='same', activation='relu')(input_layer)
x = Conv2D (filters=64, kernel_size=3, padding='same', activation='relu')(x)
x = MaxPool2D(pool_size=2, strides=2, padding='same')(x)
print(x.shape)
feat2 = GlobalAveragePooling2D()(x)
print("feature2", feat2.shape)
x = Conv2D (filters=128, kernel_size=3, padding='same', activation='relu')(x)
x = Conv2D (filters=128, kernel_size=3, padding='same', activation='relu')(x)
x = MaxPool2D(pool_size=2, strides=2, padding='same')(x)
print(x.shape)
feat3 = GlobalAveragePooling2D()(x)
print("feature3", feat3.shape)
x = Conv2D (filters=256, kernel_size=3, padding='same', activation='relu')(x) 
x = Conv2D (filters=256, kernel_size=3, padding='same', activation='relu')(x) 
#x = Conv2D (filters=256, kernel_size=3, padding='same', activation='relu')(x) 
x = MaxPool2D(pool_size=2, strides=2, padding='same')(x)
print(x.shape)
feat4 = GlobalAveragePooling2D()(x)
print("feature4", feat4.shape)
x = Conv2D (filters=512, kernel_size=3, padding='same', activation='relu')(x)
x = Conv2D (filters=512, kernel_size=3, padding='same', activation='relu')(x)
#x = Conv2D (filters=512, kernel_size=3, padding='same', activation='relu')(x)
x = MaxPool2D(pool_size=2, strides=2, padding='same')(x)
print(x.shape)
feat5 = GlobalAveragePooling2D()(x)
print("feature5", feat5.shape)
x = Conv2D (filters=512, kernel_size=3, padding='same', activation='relu')(x)
x = Conv2D (filters=512, kernel_size=3, padding='same', activation='relu')(x)
#x = Conv2D (filters=512, kernel_size=3, padding='same', activation='relu')(x)
#x = MaxPool2D(pool_size=2, strides=2, padding='same')(x)
#print(x.shape)
x = GlobalAveragePooling2D()(x)
print(x.shape)
feat6 = x
print("feature6", feat6.shape)

x = tf.concat([feat1, feat2, feat3, feat4, feat5, feat6], -1)
print("combined feature", x.shape)
x = Dense(units=4096, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(units=4096, activation='relu')(x) 
x = Dropout(0.5)(x)
output_layer = Dense(units=8, activation='softmax')(x)
model = Model(inputs=input_layer, outputs=output_layer)
model.summary()

model.compile(optimizer=adam.Adam(learning_rate=2e-4), 
              loss=mse, 
              metrics = [model_acc])

model.fit(x=tf.image.grayscale_to_rgb(training_images),
          y=training_emotions,
          batch_size=32,
          epochs=30,
          validation_data=(tf.image.grayscale_to_rgb(test_images), test_emotions))

model.compile(optimizer=adam.Adam(learning_rate=1e-4), 
              loss=mse, 
              metrics = [model_acc])

model.fit(x=tf.image.grayscale_to_rgb(training_images),
          y=training_emotions,
          batch_size=32,
          epochs=30,
          validation_data=(tf.image.grayscale_to_rgb(test_images), test_emotions))

(None, 48, 48, 3)
feature1 (None, 3)
(None, 24, 24, 64)
feature2 (None, 64)
(None, 12, 12, 128)
feature3 (None, 128)
(None, 6, 6, 256)
feature4 (None, 256)
(None, 3, 3, 512)
feature5 (None, 512)
(None, 512)
feature6 (None, 512)
combined feature (None, 1475)
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 48, 48, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 48, 48, 64)   1792        input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 48, 48, 64)   36928       conv2d[0][0]                     
__________________________________

C:\Users\Darkl\anaconda3\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py:3703: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable.debug_mode()`.
  warnings.warn(


Epoch 1/30
996/996 [==============================] - 72s 67ms/step - loss: 0.0572 - model_acc: 0.3794 - val_loss: 0.0448 - val_model_acc: 0.5470
Epoch 2/30
996/996 [==============================] - 73s 73ms/step - loss: 0.0333 - model_acc: 0.6453 - val_loss: 0.0298 - val_model_acc: 0.6810
Epoch 3/30
996/996 [==============================] - 73s 73ms/step - loss: 0.0241 - model_acc: 0.7297 - val_loss: 0.0228 - val_model_acc: 0.7393
Epoch 4/30
996/996 [==============================] - 69s 70ms/step - loss: 0.0192 - model_acc: 0.7743 - val_loss: 0.0196 - val_model_acc: 0.7585
Epoch 5/30
996/996 [==============================] - 71s 71ms/step - loss: 0.0157 - model_acc: 0.8113 - val_loss: 0.0185 - val_model_acc: 0.7803
Epoch 6/30
996/996 [==============================] - 72s 72ms/step - loss: 0.0131 - model_acc: 0.8397 - val_loss: 0.0176 - val_model_acc: 0.7887
Epoch 7/30
996/996 [==============================] - 72s 72ms/step - loss: 0.0110 - model_acc: 0.8638 - val_loss: 0.0171 - 

996/996 [==============================] - 66s 66ms/step - loss: 5.4616e-04 - model_acc: 0.9931 - val_loss: 0.0137 - val_model_acc: 0.8261
Epoch 27/30
996/996 [==============================] - 65s 66ms/step - loss: 5.3248e-04 - model_acc: 0.9924 - val_loss: 0.0136 - val_model_acc: 0.8275
Epoch 28/30
996/996 [==============================] - 66s 66ms/step - loss: 5.2359e-04 - model_acc: 0.9931 - val_loss: 0.0137 - val_model_acc: 0.8292
Epoch 29/30
996/996 [==============================] - 65s 65ms/step - loss: 5.1446e-04 - model_acc: 0.9929 - val_loss: 0.0137 - val_model_acc: 0.8286
Epoch 30/30
996/996 [==============================] - 65s 65ms/step - loss: 5.1619e-04 - model_acc: 0.9929 - val_loss: 0.0138 - val_model_acc: 0.8264


In [9]:
# VGG16 .2drop combined features
input_layer = Input(shape=(48,48,3))
print(input_layer.shape)
feat1 = GlobalAveragePooling2D()(input_layer)
print("feature1", feat1.shape)
x = Conv2D (filters=64, kernel_size=3, padding='same', activation='relu')(input_layer)
x = Conv2D (filters=64, kernel_size=3, padding='same', activation='relu')(x)
x = MaxPool2D(pool_size=2, strides=2, padding='same')(x)
print(x.shape)
feat2 = GlobalAveragePooling2D()(x)
print("feature2", feat2.shape)
x = Conv2D (filters=128, kernel_size=3, padding='same', activation='relu')(x)
x = Conv2D (filters=128, kernel_size=3, padding='same', activation='relu')(x)
x = MaxPool2D(pool_size=2, strides=2, padding='same')(x)
print(x.shape)
feat3 = GlobalAveragePooling2D()(x)
print("feature3", feat3.shape)
x = Conv2D (filters=256, kernel_size=3, padding='same', activation='relu')(x) 
x = Conv2D (filters=256, kernel_size=3, padding='same', activation='relu')(x) 
x = Conv2D (filters=256, kernel_size=3, padding='same', activation='relu')(x) 
x = MaxPool2D(pool_size=2, strides=2, padding='same')(x)
print(x.shape)
feat4 = GlobalAveragePooling2D()(x)
print("feature4", feat4.shape)
x = Conv2D (filters=512, kernel_size=3, padding='same', activation='relu')(x)
x = Conv2D (filters=512, kernel_size=3, padding='same', activation='relu')(x)
x = Conv2D (filters=512, kernel_size=3, padding='same', activation='relu')(x)
x = MaxPool2D(pool_size=2, strides=2, padding='same')(x)
print(x.shape)
feat5 = GlobalAveragePooling2D()(x)
print("feature5", feat5.shape)
x = Conv2D (filters=512, kernel_size=3, padding='same', activation='relu')(x)
x = Conv2D (filters=512, kernel_size=3, padding='same', activation='relu')(x)
x = Conv2D (filters=512, kernel_size=3, padding='same', activation='relu')(x)
x = MaxPool2D(pool_size=2, strides=2, padding='same')(x)
print(x.shape)
x = GlobalAveragePooling2D()(x)
print(x.shape)
feat6 = x
print("feature6", feat6.shape)

x = tf.concat([feat1, feat2, feat3, feat4, feat5, feat6], -1)
print("combined feature", x.shape)
x = Dense(units=4096, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(units=4096, activation='relu')(x) 
x = Dropout(0.2)(x)
output_layer = Dense(units=8, activation='softmax')(x)
model = Model(inputs=input_layer, outputs=output_layer)
model.summary()

model.compile(optimizer=adam.Adam(learning_rate=2e-4), 
              loss=mse, 
              metrics = [model_acc])

model.fit(x=tf.image.grayscale_to_rgb(training_images),
          y=training_emotions,
          batch_size=32,
          epochs=30,
          validation_data=(tf.image.grayscale_to_rgb(test_images), test_emotions))

model.compile(optimizer=adam.Adam(learning_rate=1e-4), 
              loss=mse, 
              metrics = [model_acc])

model.fit(x=tf.image.grayscale_to_rgb(training_images),
          y=training_emotions,
          batch_size=32,
          epochs=30,
          validation_data=(tf.image.grayscale_to_rgb(test_images), test_emotions))

(None, 48, 48, 3)
feature1 (None, 3)
(None, 24, 24, 64)
feature2 (None, 64)
(None, 12, 12, 128)
feature3 (None, 128)
(None, 6, 6, 256)
feature4 (None, 256)
(None, 3, 3, 512)
feature5 (None, 512)
(None, 2, 2, 512)
(None, 512)
feature6 (None, 512)
combined feature (None, 1475)
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 48, 48, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_10 (Conv2D)              (None, 48, 48, 64)   1792        input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_11 (Conv2D)              (None, 48, 48, 64)   36928       conv2d_10[0][0]                  
______________

996/996 [==============================] - 72s 72ms/step - loss: 0.0146 - model_acc: 0.8233 - val_loss: 0.0178 - val_model_acc: 0.7866
Epoch 7/30
996/996 [==============================] - 72s 72ms/step - loss: 0.0124 - model_acc: 0.8478 - val_loss: 0.0181 - val_model_acc: 0.7906
Epoch 8/30
996/996 [==============================] - 72s 72ms/step - loss: 0.0106 - model_acc: 0.8675 - val_loss: 0.0171 - val_model_acc: 0.7991
Epoch 9/30
996/996 [==============================] - 72s 72ms/step - loss: 0.0092 - model_acc: 0.8830 - val_loss: 0.0161 - val_model_acc: 0.8047
Epoch 10/30
996/996 [==============================] - 72s 72ms/step - loss: 0.0079 - model_acc: 0.8980 - val_loss: 0.0163 - val_model_acc: 0.8052
Epoch 11/30
996/996 [==============================] - 72s 72ms/step - loss: 0.0070 - model_acc: 0.9124 - val_loss: 0.0167 - val_model_acc: 0.8002
Epoch 12/30
996/996 [==============================] - 72s 72ms/step - loss: 0.0062 - model_acc: 0.9211 - val_loss: 0.0153 - val_mode

In [10]:
# VGG16 no drop/combine
input_layer = Input(shape=(48,48,3))
print(input_layer.shape)


x = Conv2D (filters=64, kernel_size=3, padding='same', activation='relu')(input_layer)
x = Conv2D (filters=64, kernel_size=3, padding='same', activation='relu')(x)
x = MaxPool2D(pool_size=2, strides=2, padding='same')(x)
print(x.shape)

x = Conv2D (filters=128, kernel_size=3, padding='same', activation='relu')(x)
x = Conv2D (filters=128, kernel_size=3, padding='same', activation='relu')(x)
x = MaxPool2D(pool_size=2, strides=2, padding='same')(x)
print(x.shape)

x = Conv2D (filters=256, kernel_size=3, padding='same', activation='relu')(x) 
x = Conv2D (filters=256, kernel_size=3, padding='same', activation='relu')(x) 
x = Conv2D (filters=256, kernel_size=3, padding='same', activation='relu')(x) 
x = MaxPool2D(pool_size=2, strides=2, padding='same')(x)
print(x.shape)

x = Conv2D (filters=512, kernel_size=3, padding='same', activation='relu')(x)
x = Conv2D (filters=512, kernel_size=3, padding='same', activation='relu')(x)
x = Conv2D (filters=512, kernel_size=3, padding='same', activation='relu')(x)
x = MaxPool2D(pool_size=2, strides=2, padding='same')(x)
print(x.shape)

x = Conv2D (filters=512, kernel_size=3, padding='same', activation='relu')(x)
x = Conv2D (filters=512, kernel_size=3, padding='same', activation='relu')(x)
x = Conv2D (filters=512, kernel_size=3, padding='same', activation='relu')(x)
x = MaxPool2D(pool_size=2, strides=2, padding='same')(x)
print(x.shape)
x = GlobalAveragePooling2D()(x)
print(x.shape)

x = Dense(units=4096, activation='relu')(x)

x = Dense(units=4096, activation='relu')(x) 

output_layer = Dense(units=8, activation='softmax')(x)
model = Model(inputs=input_layer, outputs=output_layer)
model.summary()

model.compile(optimizer=adam.Adam(learning_rate=2e-4), 
              loss=mse, 
              metrics = [model_acc])

model.fit(x=tf.image.grayscale_to_rgb(training_images),
          y=training_emotions,
          batch_size=32,
          epochs=30,
          validation_data=(tf.image.grayscale_to_rgb(test_images), test_emotions))

model.compile(optimizer=adam.Adam(learning_rate=1e-4), 
              loss=mse, 
              metrics = [model_acc])

model.fit(x=tf.image.grayscale_to_rgb(training_images),
          y=training_emotions,
          batch_size=32,
          epochs=30,
          validation_data=(tf.image.grayscale_to_rgb(test_images), test_emotions))

(None, 48, 48, 3)
(None, 24, 24, 64)
(None, 12, 12, 128)
(None, 6, 6, 256)
(None, 3, 3, 512)
(None, 2, 2, 512)
(None, 512)
Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 48, 48, 3)]       0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 48, 48, 64)        1792      
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 48, 48, 64)        36928     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 24, 24, 64)        0         
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 24, 24, 128)       73856     
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 24, 24, 128)       147584    
__

996/996 [==============================] - 69s 69ms/step - loss: 0.0012 - model_acc: 0.9921 - val_loss: 0.0147 - val_model_acc: 0.8202
Epoch 3/30
996/996 [==============================] - 69s 69ms/step - loss: 0.0010 - model_acc: 0.9925 - val_loss: 0.0146 - val_model_acc: 0.8244
Epoch 4/30
996/996 [==============================] - 69s 69ms/step - loss: 9.7084e-04 - model_acc: 0.9928 - val_loss: 0.0146 - val_model_acc: 0.8245
Epoch 5/30
996/996 [==============================] - 69s 69ms/step - loss: 9.2127e-04 - model_acc: 0.9915 - val_loss: 0.0146 - val_model_acc: 0.8256
Epoch 6/30
996/996 [==============================] - 69s 69ms/step - loss: 8.4856e-04 - model_acc: 0.9935 - val_loss: 0.0145 - val_model_acc: 0.8228
Epoch 7/30
996/996 [==============================] - 69s 69ms/step - loss: 7.8512e-04 - model_acc: 0.9934 - val_loss: 0.0147 - val_model_acc: 0.8278
Epoch 8/30
996/996 [==============================] - 69s 69ms/step - loss: 7.3493e-04 - model_acc: 0.9933 - val_loss: 

In [11]:
model.compile(optimizer=adam.Adam(learning_rate=1e-4), 
              loss=mse, 
              metrics = [model_acc])

model.fit(x=tf.image.grayscale_to_rgb(training_images),
          y=training_emotions,
          batch_size=32,
          epochs=10,
          validation_data=(tf.image.grayscale_to_rgb(test_images), test_emotions))

Epoch 1/10
996/996 [==============================] - 71s 71ms/step - loss: 3.0574e-04 - model_acc: 0.9976 - val_loss: 0.0146 - val_model_acc: 0.8284
Epoch 2/10
996/996 [==============================] - 70s 70ms/step - loss: 2.9840e-04 - model_acc: 0.9976 - val_loss: 0.0147 - val_model_acc: 0.8261
Epoch 3/10
996/996 [==============================] - 70s 70ms/step - loss: 2.9632e-04 - model_acc: 0.9975 - val_loss: 0.0146 - val_model_acc: 0.8239
Epoch 4/10
996/996 [==============================] - 70s 71ms/step - loss: 2.9840e-04 - model_acc: 0.9973 - val_loss: 0.0146 - val_model_acc: 0.8273
Epoch 5/10
996/996 [==============================] - 71s 72ms/step - loss: 2.8331e-04 - model_acc: 0.9976 - val_loss: 0.0145 - val_model_acc: 0.8287
Epoch 6/10
996/996 [==============================] - 71s 71ms/step - loss: 2.7665e-04 - model_acc: 0.9977 - val_loss: 0.0146 - val_model_acc: 0.8295
Epoch 7/10
996/996 [==============================] - 70s 70ms/step - loss: 2.7509e-04 - model_acc: 